# Binary classification using scikit-learn

In transient astronomy, difference imaging forms the basis of finding new transient sources in the sky. The idea simply being that the telescope survey has reference images of the patches of the sky and the nightly data is compared with the reference
images to search for moving/non-moving transients. Moving transients involve things like asteriods while non-moving transients involve objects like supernovae (SNe) and kilo/macro-novae.

The *search* is done by an image subtraction pipeline (ISP). What makes image subtraction non-trivial is the presence of noise (not surprising) in every pixel of the CCD. The ISP has to take into account the statistical fluctuations coming from the thermal noise of the instrument and also the fluctuations in photon count from an actual astrophysical source.

- The bottomline, barring the the procedure of image subtractions, being that the ISP either *finds* a transient in a certain field or *does not find* it.
- Suppose we wish to find the conditions when the ISP *finds*/*doesn't find* a transient.
- One way (maybe the only way) to do this is to inject fake transients and then run the ISP.
- In this case, you know the properties of a transient you are injecting. So you *know* for which cases the ISP found the transient.

**But running the ISP is computationally expensive, time consuming, and most importantly, one should know how to run it!**


So it is advantageous to have *something* that can mimic the behavior of the ISP. After all, we know roughly how the ISP should behave
- It should find bright transients.
- It should not find transients if the weather conditions are poor.
- It should not find transients if the transient is in a bright galaxy.

Machine learning will help us build such a decision-maker. Note that, we don't all the properties of the pipeline. We simpy want the bottomline - was the transient *found*/*not found*.

Let's start by importing some libraries.
- `pickle`: a format to store data/objects in a compressed form in python.
- `numpy`: python array/vector operations library. Whenever you are writing a loop in python, double check the docs if it can be vectorized.
- `pandas`: this is essential data origanization tool. Not native to `python`, used heavily in statistical tools like `R`. Gives easy infrastructure to organize, plot and sample (large) datasets.
- `matplotlib`: needs no description!
- `seaborn`: maybe needs a description :) Plotting library, customized for data analysis. Distributions, getting smoothened verions of the same, pair-wise dependence of variables in data are very easily seen. I have come to love it with time.

In [ ]:
import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams["figure.figsize"] = (12, 10)

Imagine we already have (saving a couple of months) the set of fake injections which were recovered/missed from an ISP. Let's load the data

In [ ]:
with open('../data/found_injections.pkl', 'rb') as f, \
        open('../data/all_injections.pkl', 'rb') as g:
    df_found_complete = pickle.load(f)
    df_all_complete = pickle.load(g)

Take a look at your dataset

In [ ]:
df_all_complete.head()

We are not interested in the complete dataset. Let us select a subset of the columns that will be pertinent to us. Also, Let us define another dataframe which will store all the missed injections.

Dataframes have useful methods for filtering. This is similar to numpy indexing with added benefits!
The `.loc` method (not sure if I should call it a method since one calls it with `[...]` instead of `(...)`. Anyway, see [docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html)) takes in two arguments:
- The first one is a boolean which says which items to choose
- The second is a list of columns to filter out

I have also used one of the many useful `numpy` functions for vector operations. The `np.in1d` method (see [docs](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.in1d.html)) will take in two arrays and return an array of length of the *first* argument with boolean values depending on whether the elements of the second array are present in the first

In [ ]:
relevant_cols = ['stamp_mag', 'lmt_mag', 'surface_brightness', 'seeing', 'medsky']

df_all = df_all_complete.loc[:, relevant_cols]
df_found = df_found_complete.loc[:, relevant_cols]

df_missed = df_all_complete.loc[
    ~np.in1d(df_all_complete['stamp_id'].values, df_found_complete['stamp_id'].values),
    relevant_cols
]

In [ ]:
df_all.head()

The prefix *all*, *missed* and *found* are the set of all, missed and found fake transients respectively.
The column column descriptions are as follows:
- `stamp_mag` is the magnitude of the transient
- `lmt_mag` is the limiting magnitude of the instrument
- `surface_brightness` is the surface brightness of the galaxy when the transient is simulated
- `seeing` is the astronomical seeing at that epoch. It increases with the increasing turbulence in the atmosphere. 
- `medsky` is the median sky counts, a measure of sky brightness

Generally, it is the FWHM of the point spread function of the telescope, measured in pixel units.

### Fake transients magnitudes distribution
Let's have a look at what fake transients we injected and what was found

In [ ]:
plt.figure()
plt.hist(df_all['stamp_mag'], bins=20, histtype='stepfilled', color='black', alpha=0.3, label='All')
plt.hist(df_found['stamp_mag'], bins=20, histtype='stepfilled', color='cyan', alpha=0.8, label='Found')
plt.hist(df_missed['stamp_mag'], bins=20, histtype='stepfilled', color='red', alpha=0.5, label='Missed')
plt.legend()
plt.xlabel(r'$m_{\mathrm{inj}}$', fontsize=15)
plt.ylabel(r'Counts')
plt.grid(linestyle='--')

A word about magnitudes. This is basically a log-scale for flux of a source, defined as:
\begin{equation}
    m = -2.5 \log_{10}\left(\frac{F}{F_0}\right)
\end{equation}
where $F$ is the flux in particular filter while $F_0$ is some reference flux usually chosen with respect to some standard bright stars. The logarithmic scale is simply to take into account the fact that our perception follows such a measure. Think of the analogous definition of *loudness* (in units of decibel):
\begin{equation}
    L = 10 \log_{10}\left(\frac{I}{I_0}\right)
\end{equation}
where $I$ is the sound intensity and $I_0$ is threshold of hearing.

Coming back to magnitudes, this is simply a number (no units). Brightest stars on sky have magnitude $\simeq 1$, dimmest stars, visible to naked eye) have magnitude $\simeq 5$. The magnitudes we are dealing with $\simeq 20$, which is almost a factor of $10^6$ fainter in flux than the faintest we can see with our naked eye. No wonder telescopes are expensive!

### Pairplots
The above plot is simply the transients binned up in the transient magnitude. But, clearly, we have so many parameters which tell us about the observing conditions. The above histogram says that as the transient becomes dim, it is missed by the ISP. We could do the same distribution for galaxy surface brightness to check the dependence.

How do we see them all together? Visualizing anything ($\geq 3$)-dimensional is hard. We could, however, make *pairplots* to see the dependence. This is where `seaborn` makes our life an order of magnitude easier!

Before doing that let's add a column to the `df_all` dataframe called `found` and fill it out with boolean values. After doing this, clearly, we don't need to work with the two other dataframe `df_found`, `df_missed`.

In [ ]:
# add a new column and use the same in1d as used above to find the indices of found transients
df_all = df_all.assign(found=np.in1d(df_all_complete['stamp_id'].values, df_found_complete['stamp_id'].values))
df_all.head()

In [ ]:
g = sns.pairplot(df_all,
                 vars=['stamp_mag', 'lmt_mag', 'surface_brightness', 'seeing'],
                 hue='found',
                 diag_kind='auto')
# this loop is to make the upper triangle invisible, since it is simply a redundancy.
# feel free to remove to get the upper triangle back
for i, j in zip(*np.triu_indices_from(g.axes, 1)):
    g.axes[i, j].set_visible(False)

# The following replace the labels to something custom
replacements = {'stamp_mag': r'$m_{\mathrm{inj}}$', 'lmt_mag': r'$m_{\mathrm{lim}}$',
                'surface_brightness': r'$S_{\mathrm{gal}} \mathrm{mag (arcsec)^{-2}}$',
                'seeing': r'Seeing'}
for i in range(4):
    for j in range(4):
        xlabel = g.axes[i][j].get_xlabel()
        ylabel = g.axes[i][j].get_ylabel()
        if xlabel in replacements.keys():
            g.axes[i][j].set_xlabel(replacements[xlabel])
        if ylabel in replacements.keys():
            g.axes[i][j].set_ylabel(replacements[ylabel])

Plots like these help us identify whether there is a *boundary* in the data, or if there is some sort of clustering into groups. In this case, we can clearly see that there is some sort of boundary for plots for $m_{\mathrm{inj}}$ and any other parameter.

## Onto Machine Learning
Let's see how well the machine is able to resolve these boundaries. 

There are a suite of supervised classifiers that `sklearn` provides which vary in speed and complexity. For my research, where the dataset was much larger, I have used `KNeighborsClassifier` classifier. This classifier would create a [Voronoi tessellation](https://en.wikipedia.org/wiki/Voronoi_diagram) in the parameter space about the points and make predictions on the cases you wish to test using values from the trained nearest neighbors. This type of classification models are called *non-parameteric* i.e. no model/fitting parameters involved. The decision algorithm is simple and it is computationally fast.

Let's start by some imports.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

Create an instance of the classifier by supplying it the number of neighbors you would like to consider. I typically like to start using something like $2N + 1$, $N$ being the the number of dimensions/features in the problem. The notion simply being roughly there are 2 points for every dimension and one more to break a tie.

In [ ]:
# For now we have 4 features, so let's use 9 neighbors
num_neighbors = 9
KNC = KNeighborsClassifier(num_neighbors)

In [ ]:
X_train = df_all[['stamp_mag', 'lmt_mag', 'surface_brightness', 'seeing']].values
y_train = df_all['found'].values

X_train.shape, y_train.shape

Use the `fit` method, generic to all `sklearn` models, to initialiaze the machine with the training data.

In [ ]:
KNC.fit(X_train, y_train)

Use the `predict` method to make predictions now on any arbitrary parameter set. This simply gives the decision based on a majority vote across the K neighbors. Use the `predict_proba` method to get a probability, which is essentially, the fraction of K neighbors which fall in one class.

In [ ]:
# the following is the prediction and the probability of found for a arbitrary parameter set, say,
# stamp_mag=19, lmt_mg=20, surface_brightness=20, seeing=20
stamp_mag=19.5
lmt_mg=20
sb=20
seeing=2

KNC.predict([[stamp_mag, lmt_mg, sb, seeing]]), KNC.predict_proba([[stamp_mag, lmt_mg, sb, seeing]]).T[1]

That's essentially it!

Clearly, more the number, more the computation time. Also, more the neighbors, more the bias in training. The *sweet spot* is only checked by trying out a few numbers and by cross-validation.

## Testing and Cross-validation

In order to evaluate the inaccuracy of the classifier, we could split our full dataset into training and testing samples. Say, we train on $90\%$ of the dataset and test on $10\%$. For the latter we can compare the true answer to the result that machine gives us.

There are many submodules for testing and cross-validation in `sklearn`, let's use `train_test_split` which conveniently splits the data. Also we will repeat the entire process multiple times, say 10 times, to check for consistency.

In [ ]:
# assigning new variables to the data 
features = df_all[['stamp_mag', 'lmt_mag', 'surface_brightness', 'seeing']].values
targets = df_all['found'].values

# let's check an example of the training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.1)
plt.hist((X_train.T[0], X_test.T[0]), bins=30, histtype='step', color=('black', 'red'))
plt.grid(linestyle='--')
plt.xlabel(r'$m_{\mathrm{inj}}$', fontsize=15)
plt.show()

In [ ]:
# running train_test_split
for ii in range(10):
    X_train, X_test, y_train, y_test = train_test_split(features, targets, \
                                                        test_size=0.1, random_state=ii)
    KNC = KNeighborsClassifier(num_neighbors)
    KNC.fit(X_train, y_train)
    KNC_predict = KNC.predict(X_test)
    
    incorrect = np.logical_xor(KNC_predict, y_test)
    correct = np.logical_not(incorrect)
    incorrect = np.sum(incorrect)
    correct = np.sum(correct)
    percentage_correct = 100.*np.float(correct)/(correct+incorrect)
    percentage_incorrect = 100.*incorrect/(correct+incorrect)
    
    print(
        "Correct predictions {}, incorrect predictions {}, correct {}%, incorrect {}%".format(
        correct,
        incorrect,
        percentage_correct,
        percentage_incorrect))

## Including correlations between parameters

Sometimes the parameters that categorize the data are correlated. Let's use a different set of features and make a pairplot.

In [ ]:
correlated_parameters = ['stamp_mag', 'lmt_mag', 'medsky']
g = sns.pairplot(df_all,
                 vars=correlated_parameters,
                 hue='found',
                 diag_kind='auto')

for i, j in zip(*np.triu_indices_from(g.axes, 1)):
    g.axes[i, j].set_visible(False)

There is a trend in the plot between the `lmt_mag` and `medsky`. Recall, that `medsky` is the sky brightness which intuitively should lead to a poor limiting magnitude. This is seen in the plot where increasing values of limiting magnitude correlates with a dimmer sky. It would help if we could convey this information to the machine.

This can be done using the [Mahalanobis distance](https://en.wikipedia.org/wiki/Mahalanobis_distance). The basic idea is to change the metric in the feature space from *Euclidean* to something which takes correlations into account a.k.a. *mahalanobis* metric. This is done automatically by the classifier given the covariance matrix.

In [ ]:
features = df_all[correlated_parameters]
targets = df_all.found

In [ ]:
features.head()

In [ ]:
cov = features.cov()
cov

Let's run `train_test_split` by modifying and running the classifier with mahalanobis metric. (This might take time)

In [ ]:
for ii in range(5):
    X_train, X_test, y_train, y_test = train_test_split(features, targets, \
                                                        test_size=0.1, random_state=ii)
    cov = X_train.cov()
    inv_cov = np.linalg.inv(cov)
    KNC = KNeighborsClassifier(num_neighbors,
                               metric='mahalanobis',
                               metric_params={'VI': inv_cov}, n_jobs=-1)
    KNC.fit(X_train, y_train)
    KNC_predict = KNC.predict(X_test)

    incorrect = np.logical_xor(KNC_predict, y_test)
    correct = np.logical_not(incorrect)
    incorrect = np.sum(incorrect)
    correct = np.sum(correct)
    percentage_correct = 100.*np.float(correct)/(correct+incorrect)
    percentage_incorrect = 100.*incorrect/(correct+incorrect)
    
    print(
        "Correct predictions {}, incorrect predictions {}, correct {}%, incorrect {}%".format(
        correct,
        incorrect,
        percentage_correct,
        percentage_incorrect))

And once with the standard euclidean metric (note that we have changes the features from last time)

In [ ]:
for ii in range(10):
    X_train, X_test, y_train, y_test = train_test_split(features, targets, \
                                                        test_size=0.1, random_state=ii)
    KNC = KNeighborsClassifier(num_neighbors, n_jobs=-1)
    KNC.fit(X_train, y_train)
    KNC_predict = KNC.predict(X_test)

    incorrect = np.logical_xor(KNC_predict, y_test)
    correct = np.logical_not(incorrect)
    incorrect = np.sum(incorrect)
    correct = np.sum(correct)
    percentage_correct = 100.*np.float(correct)/(correct+incorrect)
    percentage_incorrect = 100.*incorrect/(correct+incorrect)
    
    print(
        "Correct predictions {}, incorrect predictions {}, correct {}%, incorrect {}%".format(
        correct,
        incorrect,
        percentage_correct,
        percentage_incorrect))